## 8.2 딥드림

### 8.2.1 케라스 딥드림 구현

In [4]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

# 모든 훈련 연산 비활성화
K.set_learning_phase(0) 
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [8]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

### 코드 8-9 딥드림 설정

In [9]:
layer_contributions = {'mixed2':0.2,
                      'mixed3':3.,
                      'mixed4':2.,
                      'mixed5':1.5,}

### 코드 8-10 최대화할 손실 정의

In [10]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: Tensor("Const:0", shape=(1,), dtype=int32) must be from the same graph as Tensor("Cast_4:0", shape=(4,), dtype=float32).

### 코드 8-11 경사 상승법 과정

In [14]:
dream = model.input 

grads = K.gradients(loss, dream)[0]

# 그래디언트 정규화
grads = grads / K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grads_values = outs[1]
    return loss_value, grad_values

# 경사 상승법
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x

ValueError: Tensor("input_1:0", shape=(None, None, None, 3), dtype=float32) must be from the same graph as Tensor("Variable/Read/ReadVariableOp:0", shape=(), dtype=float32).

### 코드 8-12 연속적인 스케일러 걸쳐 경사 상승법 실행

In [12]:
import numpy as np

# 경사 상승법 단계 크기
step = 0.01 

# 경사 상승법을 실행할 스케일 단계 횟수
num_octave = 3 

# 스케일 간 비율
octave_scale = 1.4 

 # 스케일 단계 마다 수행할 경사 상승법 횟수
iterations = 20

#이상한 그림이 되는 것을 방지 위해, 손실이 10보다 커지면 경사 상승법 과정 중지
max_loss = 10.

base_image_path = "./datasets/dragon.jpg"

img = preprocess_image(base_image_path) # 기본 이미지를 넘파이 배열로 로드

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0]) # 이미지 넘파이 배열을 가장 작은 스케일로 변경

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, 
                         iterations=iterations,
                         step=step,
                         max_loss=max_loss)
    
    unscaled_shrunk_original_img = resize_img(shrunk_original_img, shape) # 작게 줄인 원본 이미지의 스케일을 높임. 픽셀의 경계가 보일 것임
    
    same_size_original = resize_img(original_img, shape) #이 크기에 해당하는 원본 이미지의 고해상도 버전을 계산
        
    lost_detail = same_size_original - unscaled_shrunk_original_img # 두 이미지의 차이가 스케일을 높였을 때 손실된 디테일
    
    img += loss_detail # 손실된 디테일을 딥드림 이미지에 다시 주입
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='./datasets/final_dream.png')

처리할 이미지 크기 (268, 358)


NameError: name 'gradient_ascent' is not defined

### 코드 8-13 유틸리티 함수

In [13]:
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
              float(size[0]) / img.shape[1],
              float(size[1]) / img.shape[2],
              1)
    
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transposed((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x